In [18]:
import pandas as pd
import numpy as np
from scipy import stats, optimize
from scipy.special import gamma

## Data ingestion

In [19]:
credit = pd.read_excel("qrm25HSG_creditportfolio.xlsx")
idx_raw = pd.read_excel("qrm25HSG_indexes.xlsx")

prices = (
    pd.DataFrame({
        "Date": idx_raw["SPI Index"],
        "SPI": pd.to_numeric(idx_raw["Unnamed: 1"], errors="coerce"),
        "SPX": pd.to_numeric(idx_raw["Unnamed: 3"], errors="coerce"),
    })
    .dropna()
    .set_index("Date")
)

# ---------------------------------------------------------
# Get the logret_weekly array
# ---------------------------------------------------------

logret_daily = np.log(prices / prices.shift(1)).dropna()
logret_weekly = logret_daily.resample("W-FRI").sum().dropna()
theta_hist = logret_weekly.values     # shape (T, 2)
T = theta_hist.shape[0]

In [20]:
# ---------------------------------------------------------
# (iv) M2: 
# ---------------------------------------------------------
mu_M2 = theta_hist.mean(axis=0)                 # shape (2,)
Sigma_M2 = np.cov(theta_hist.T, bias=True)      # shape (2,2)

print("M2 (Gaussian) estimates")
print("mu_M2 =", mu_M2)
print("Sigma_M2 =\n", Sigma_M2)

# ---------------------------------------------------------
# (iv) M3: 
# ---------------------------------------------------------

#To be completed

M2 (Gaussian) estimates
mu_M2 = [0.00129291 0.00196092]
Sigma_M2 =
 [[0.00052562 0.0004085 ]
 [0.0004085  0.00059622]]


In [21]:
# ---------------------------------------------------------
# Common: compute s_k and d_k for all k
# ---------------------------------------------------------
Sigma_emp = np.cov(theta_hist.T, bias=True)        # empirical Cov(Θ)

a = credit[["a_k1", "a_k2"]].values                # shape (K,2)
s2_k = np.einsum("ki,ij,kj->k", a, Sigma_emp, a)   # a_k' Σ a_k
s_k = np.sqrt(s2_k)

credit["s_k"] = s_k
credit["d_k"] = credit["s_k"] * stats.norm.ppf(credit["pi_k"])

Simulate values of $Y_k$ for each model

In [22]:
# ---------------------------------------------------------
# Helper: simulate Theta under each model
# ---------------------------------------------------------

def simulate_theta_M1(n_sims, theta_hist, rng):
    idx = rng.integers(0, len(theta_hist), size=n_sims)
    return theta_hist[idx, :]   # shape (n_sims, 2)


def simulate_theta_M2(n_sims, mu, Sigma, rng):
    return rng.multivariate_normal(mean=mu, cov=Sigma, size=n_sims)

def simulate_theta_M3():
    pass


# ---------------------------------------------------------
# Helper: simulate Y
# ---------------------------------------------------------
lam = credit["lambda_k"].values
s_k = credit["s_k"].values    

K = len(credit)
a = credit[["a_k1", "a_k2"]].values

def simulate_Y(theta_sims, rng):

    n_sims = theta_sims.shape[0]

    # systematic part: sqrt(λ_k) * a_k' Θ
    systematic_base = theta_sims @ a.T           # (n_sims, K)
    sqrt_lambda = np.sqrt(lam)[None, :]         # (1,K) broadcast
    systematic = sqrt_lambda * systematic_base  # (n_sims, K)

    # idiosyncratic part: sqrt(1-λ_k) * s_k * ε_k
    eps = rng.standard_normal(size=(n_sims, K))
    idio = np.sqrt(1 - lam)[None, :] * s_k[None, :] * eps

    Y = systematic + idio
    return Y

In [23]:
# ---------------------------------------------------------
# Run each simulation
# ---------------------------------------------------------

N_SIM = 10_000
rng = np.random.default_rng(123) # Keep this for replicability!


# ---------------------------------------------------------
# M1:
# ---------------------------------------------------------

theta_M1 = simulate_theta_M1(N_SIM, theta_hist, rng)
Y_M1 = simulate_Y(theta_M1, rng)   # shape (N_SIM, K)

print("\nM1: Y simulations shape =", Y_M1.shape)
print("Example: first scenario Y_M1[0, :5] =", Y_M1[0, :5])
print("Thresholds d_k already stored in credit['d_k'].")

# ---------------------------------------------------------
# M2:
# ---------------------------------------------------------

theta_M2 = simulate_theta_M2(N_SIM, mu_M2, Sigma_M2, rng)
Y_M2 = simulate_Y(theta_M2, rng)

print("\nM2: Y simulations shape =", Y_M2.shape)
print("Example: first scenario Y_M2[0, :5] =", Y_M2[0, :5])


# ---------------------------------------------------------
# M3:
# ---------------------------------------------------------

theta_M3 = simulate_theta_M3()
#Y_M3 = simulate_Y(theta_M3, rng)
#print("\nM3: Y simulations shape =", Y_M3.shape)
#print("Example: first scenario Y_M3[0, :5] =", Y_M3[0, :5])


M1: Y simulations shape = (10000, 100)
Example: first scenario Y_M1[0, :5] = [-0.00691078  0.03794499  0.01008346 -0.00442683  0.00780488]
Thresholds d_k already stored in credit['d_k'].

M2: Y simulations shape = (10000, 100)
Example: first scenario Y_M2[0, :5] = [ 0.0128842  -0.01097461 -0.01832065 -0.03148574 -0.00210962]


## Derive portfolio loss distribution

In [24]:
# Basic portfolio info
E = credit["Exposure USD"].values      # (K,)
R = credit["R_k"].values               # (K,)
d = credit["d_k"].values               # (K,)
K = len(credit)

# ---------------------------------------------------------
# Helper: from Y-simulations to loss simulations
# ---------------------------------------------------------
def losses_from_Y(Y, d, E, R):
    """
    Y : (n_sims, K) simulated latent variables
    d : (K,) default thresholds
    E : (K,) exposures
    R : (K,) recovery rates in default

    Returns:
        losses : (n_sims,) simulated *portfolio losses*
                 L = sum_k E_k (1 - R_k) 1{Y_k <= d_k}
    """
    Y = np.asarray(Y)
    d = np.asarray(d)
    E = np.asarray(E)
    R = np.asarray(R)

    # Default indicators: I_{jk} = 1{Y_jk <= d_k}
    I = (Y <= d[None, :]).astype(float)          # (n_sims, K)

    # Scenario losses: L_j = sum_k E_k (1 - R_k) I_{jk}
    loss_per_name = E[None, :] * (1.0 - R[None, :]) * I
    L = loss_per_name.sum(axis=1)                # (n_sims,)
    return L

# ---------------------------------------------------------
# Compute loss simulations under the three models
# ---------------------------------------------------------
L_M1 = losses_from_Y(Y_M1, d, E, R)
L_M2 = losses_from_Y(Y_M2, d, E, R)
#L_M3 = losses_from_Y(Y_M3, d, E, R)

print("Loss sims (M1)  mean, std =", L_M1.mean(), L_M1.std())
print("Loss sims (M2)  mean, std =", L_M2.mean(), L_M2.std())
#print("Loss sims (M3)  mean, std =", L_M3.mean(), L_M3.std())


Loss sims (M1)  mean, std = 72160.22644508207 179357.7619433644
Loss sims (M2)  mean, std = 70097.10092213069 100185.88490453549


In [25]:
# ---------------------------------------------------------
# Risk measure helpers: VaR and ES for loss distributions
# ---------------------------------------------------------

def var_empirical(losses, alpha=0.99):
    losses = np.asarray(losses)
    return np.quantile(losses, alpha)


def es_empirical(losses, alpha=0.99):
    pass


# ---------------------------------------------------------
# Compute and print VaR/ES for the three models
# ---------------------------------------------------------
alphas = (0.95, 0.99, 0.999)

print("\n=== VaR / ES comparison (losses) ===")

# TO complete


=== VaR / ES comparison (losses) ===
